<a href="https://colab.research.google.com/github/marcovisintin1998/COVID-19/blob/master/evt_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyextremes
import math

import numpy as np
import pandas as pd
import pyextremes as pyxt
from matplotlib import pyplot as plt
from pyextremes import get_extremes, get_return_periods

df = pd.read_excel(r'/content/Historical_NAVs_LU0922333322.xlsx', header=2)
rows = df.count()[0]

for i in range(0, rows):
    df.iloc[i, 0] = df.iloc[i, 0].date()

"definisco vg la variazione giornaliera e scarto la prima osservazione (ha valore null)"

dataframe=df[1:rows-1]
def_series=dataframe.sort_values(by=['Data'])

vg=def_series['Prezzo (EUR)'].diff()
def_series.insert(2, "Variazione", vg, True)

"preparo i dati da usare"
fig, ax = plt.subplots(2, sharex=True)
x = def_series.Data
y1 = def_series['Prezzo (EUR)']
y2= def_series['Variazione']

"setto colori diversi per profits e losses giornalieri"
my_color = np.where(y2 >= 0, 'green', 'red')

"plot della serie del NAV e delle sue variazioni giornaliere"
ax[0].plot(x, y1,linewidth=0.6,color='blue')
ax[0].set_title('Historical NAVs')
ax[1].vlines(x=x, ymin=0, ymax=y2, color=my_color, alpha=0.4)
ax[1].scatter(x, y2, color=my_color, s=1, alpha=1)
ax[1].set_title('NAV variation')

"linea di inizio covid e dello 0 per la variazione"
"il 30/01 è la data in cui il covid è stato dichiarato problema per la salute e in cui c'è stata la Brexit, pandemia nel 11/03"
ax[0].axvline(x=pd.Timestamp('2020-01-30'),linewidth=1,color='orange')
ax[1].axvline(x=pd.Timestamp('2020-01-30'),linewidth=1,color='orange')
ax[0].text(pd.Timestamp('2020-03-11'),20, 'Covid start', fontsize = 8)
ax[1].text(pd.Timestamp('2020-03-11'),1, 'Covid start', fontsize = 8)
ax[1].text(pd.Timestamp('2020-01-30'),1.4, '2020-01-30', fontsize = 10)
ax[1].axhline(y=0,linewidth=1,color='black')
"rende la data obliqua"
fig.autofmt_xdate()

plt.show()

"cambio di segno per concentrarmi sulle perdite maggiori in senso assoluto, poi trovo VaR e ES"
losses=-y2

"VaR a 3 livelli: primo quantile t.c. fdr>= al livello dato"
losses_VaR=np.sort(losses)
n_90=math.ceil(0.9*rows)
var_90=losses_VaR[n_90]
n_95=math.ceil(0.95*rows)
var_95=losses_VaR[n_95]
n_99=math.ceil(0.99*rows)
var_99=losses_VaR[n_99]

"ES a 3 livelli: valore atteso delle perdite condizionatamente ad aver superato il rispettivo VaR"
es_90=sum(losses_VaR[losses_VaR>=var_90])/(rows-n_90)
es_95=sum(losses_VaR[losses_VaR>=var_95])/(rows-n_95)
es_99=sum(losses_VaR[losses_VaR>=var_99])/(rows-n_99)
print(var_90,es_90,var_95,es_95,var_99,es_99,np.min(losses),np.max(losses))

"modello EVT, per requirements del pacchetto deve essere series (trasformazione che avviene grazie a squeeze(), con datetime come index (è stata aggiunta già nell'excel la colonna di variazione)"
series = (pd.read_excel(r'/content/Historical_NAVs_LU0922333322.xlsx',header=2,usecols="A,C", index_col=0, parse_dates=True).sort_index(ascending=True)).astype(float).dropna().squeeze()
series=-series[1:rows-1]

"creiamo il modello (non lo fitta, è solo per settare l'ambiente di lavoro), si estraggono i massimi con il metodo block maxima in periodi di 31 giorni che verranno già associati al modello, infine il plot dei valori estremi estratti"
model=pyxt.EVA(series)
model.get_extremes(method="BM", block_size="31.2425D")
model.plot_extremes()
plt.show()

"fit del modello"
model.fit_model()

extremes = get_extremes(
    ts=series,
    method="BM",
    block_size="31.2425D",
)
return_periods = get_return_periods(
    ts=series,
    extremes=extremes,
    extremes_method="BM",
    extremes_type="high",
    block_size="31.2425D",
    return_period_size="31.2425D",
    plotting_position="weibull",
)
print(return_periods.sort_values("return period", ascending=False).head())

summary = model.get_summary(
    return_period=[1, 5, 10, 25, 50, 100],
    alpha=0.95,
    n_samples=100
)
print(summary)
summary = model.get_summary(return_period=[1, 10, 25, 50, 100],alpha=0.95,n_samples=1000)
print(summary)

model.plot_diagnostic(alpha=0.95)
plt.show()